In [ ]:
# 1. 데이터 로드
# 2. 데이터 구조 및 기초 통계 확인
# 3. 결측치 및 이상치 탐색
# 4. 데이터 시각화를 통한 탐색
# 5. 데이터 정제 및 전처리
# 6. 데이터 변환 및 피처 엔지니어링
# 7. 데이터 분할

In [ ]:
"""
1  price 
2  neighbourhood_cleansed 
3  latitude 
4  longitude 
5  property_type 
6  room_type 
7  accommodates 
8  bathrooms 
9  bedrooms 
10 beds 
11 amenities 
12 number_of_reviews 
13 number_of_reviews_ltm 
14 number_of_reviews_l30d 
15 number_of_reviews_ly 
16 first_review 
17 last_review 
18 review_scores_rating 
19 review_scores_accuracy 
20 review_scores_cleanliness 
21 review_scores_checkin 
22 review_scores_communication 
23 review_scores_location 
24 review_scores_value
"""

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 한글 폰트 사용을 위한 설정
import matplotlib.font_manager as fm
import matplotlib

font_path = 'C:\\Windows\\Fonts\\gulim.ttc'
font = fm.FontProperties(fname=font_path).get_name()
matplotlib.rc('font', family=font)

### # 특이사항 메모



---

### 주제탐색1 - 날짜별 가격변화를 예측?

##### 데이터 구조 탐색

- 날짜별 가격변화를 예측하면 성수기 여부에 따른 가격을 예측 가능

In [ ]:
# 날짜별 정보

df_calendar = pd.read_csv('./data/calendar.csv.gz', compression='gzip')

In [ ]:
# .info()를 통해 '숙소 고유 식별번호', '날짜', '가격', '가격 변동'등이 존재 함을 확인

df_calendar.info()

_="""
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8306219 entries, 0 to 8306218
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64      # 👈 숙소 고유 식별번호
 1   date            object     # 👈 날짜
 2   available       object     # 예약 가능 여부
 3   price           object     # 👈 가격
 4   adjusted_price  float64    # 👈 할인이나 프로모션 등이적용된 가격
 5   minimum_nights  float64    # 해당 날짜에 숙소를 예약하기 위해 필요한 최소 숙박 일수
 6   maximum_nights  float64    # 해당 날짜에 숙소를 예약할 수 있는 최대 숙박 일수
dtypes: float64(3), int64(1), object(3)
memory usage: 443.6+ MB
"""

In [ ]:
# 컬럼 'df_calendar['price']'가 object 타입이고 실제 데이터가 '$4,725.00' 와 같이 들어 있음을 발견하려 float형 컬럼 추가

# 'price_float' 컬럼에서 '$'와 ',' 제거
df_calendar['price_float'] = df_calendar['price'].str.replace('$', '', regex=False).str.replace(',', '', regex=False)

# 컬럼 타입 변경
df_calendar['price_float'] = df_calendar['price_float'].astype(float)

In [ ]:
# df_calendar.loc[df_calendar['listing_id'] == 197677].describe()
# 가격변화를 보기위해 숙소 최빈값을 검색

print(df_calendar['listing_id'].mode())
_="""결과
0                 4690684       # 👈 분석대상
1                 5738050       # 👈 분석대상
...
51    1342364577456789638
Name: listing_id, dtype: int64
"""

In [ ]:
# 최빈값 2개에 대하여 ".describe()" 실행했지만 가격 변동이 없었음

df_calendar.loc[df_calendar['listing_id'] == 4690684].describe()

df_calendar.loc[df_calendar['listing_id'] == 5738050].describe()

_="""                                                                  [👇]
       listing_id  adjusted_price  minimum_nights  maximum_nights  price_float
count       366.0             0.0           365.0           365.0        366.0
mean    4690684.0             NaN            14.0           120.0       4725.0
std           0.0             NaN             0.0             0.0          0.0
min     4690684.0             NaN            14.0           120.0       4725.0
25%     4690684.0             NaN            14.0           120.0       4725.0
50%     4690684.0             NaN            14.0           120.0       4725.0
75%     4690684.0             NaN            14.0           120.0       4725.0
max     4690684.0             NaN            14.0           120.0       4725.0
       listing_id  adjusted_price  minimum_nights  maximum_nights  price_float
count       366.0             0.0           365.0           365.0        366.0
mean    5738050.0             NaN             3.0          1125.0       5500.0
std           0.0             NaN             0.0             0.0          0.0
min     5738050.0             NaN             3.0          1125.0       5500.0
25%     5738050.0             NaN             3.0          1125.0       5500.0
50%     5738050.0             NaN             3.0          1125.0       5500.0
75%     5738050.0             NaN             3.0          1125.0       5500.0
max     5738050.0             NaN             3.0          1125.0       5500.0
"""

##### 주제탐색1 - 결론
- 불가 판단 -> 데이터셋을 변경

---

### 주제탐색2 - 숙소의 여러 조건에 따른 숙소 가격 예측

##### # 1. 데이터 로드

In [ ]:

df_listings = pd.read_csv('./data/listings.csv.gz', compression='gzip')

##### # 2. 데이터 구조 및 기초 통계 확인

In [ ]:
# 데이터 구조 분석
df_listings.info()

_="""
데이터 총: 25,297
컬럼 총: 79개
"""

# df_listings.describe()

In [ ]:
# 전체 컬럼 리스트
# 회의롤 통해 1차적으로 필요/불필요 컬럼을 구별
# 주석처리 = 필요한 컬럼

drop_list = [
'id',	                                        # 각 숙소의 고유 식별자.
'listing_url',	                                # 숙소의 웹페이지 URL.
'scrape_id',	                                # 데이터 스크래핑 작업의 고유 식별자.
'last_scraped',	                                # 숙소 정보가 마지막으로 스크래핑된 날짜.
'source',	                                    # 숙소 정보의 출처.
'name',	                                        # 숙소의 이름.
'description',	                                # 숙소에 대한 상세 설명.
'neighborhood_overview',	                    # 숙소 주변 지역에 대한 설명.
'picture_url',	                                # 숙소 대표 사진의 URL.
'host_id',	                                    # 호스트의 고유 식별자.
'host_url',	                                    # 호스트의 웹페이지 URL.
'host_name',	                                # 호스트의 이름.
'host_since',	                                # 호스트가 에어비앤비에 가입한 날짜.
'host_location',	                            # 호스트의 위치.
'host_about',	                                # 호스트에 대한 자기소개.
'host_response_time',	                        # 호스트의 평균 응답 시간.
'host_response_rate',	                        # 호스트의 응답률.
'host_acceptance_rate',	                        # 호스트의 예약 수락률.
'host_is_superhost',	                        # 호스트가 슈퍼호스트인지 여부.
'host_thumbnail_url',	                        # 호스트 프로필 썸네일 이미지의 URL.
'host_picture_url',	                            # 호스트 프로필 사진의 URL.
'host_neighbourhood',	                        # 호스트가 거주하는 지역.
'host_listings_count',	                        # 해당 호스트가 보유한 전체 숙소 수.
'host_total_listings_count',	                # 해당 호스트가 소유하거나 관리하는 전체 숙소 수.
'host_verifications',	                        # 호스트의 인증 정보 목록.
'host_has_profile_pic',	                        # 호스트가 프로필 사진을 가지고 있는지 여부.
'host_identity_verified',	                    # 호스트 신원이 인증되었는지 여부.
'neighbourhood',	                            # 숙소가 위치한 지역의 이름.

# 'neighbourhood_cleansed',	                    # 정제된 숙소 지역 이름.

'neighbourhood_group_cleansed',	                # 정제된 숙소 지역 그룹 이름.

# 'latitude',	                                    # 숙소의 위도.
# 'longitude',	                                # 숙소의 경도.
# 'property_type',	                            # 숙소의 종류.
# 'room_type',	                                # 숙소의 방 유형.
# 'accommodates',	                                # 숙소가 수용 가능한 최대 인원 수.
# 'bathrooms',	                                # 숙소의 욕실 개수.

'bathrooms_text',	                            # 숙소의 욕실 개수를 설명하는 텍스트.

# 'bedrooms',	                                    # 숙소의 침실 개수.
# 'beds',	                                        # 숙소의 침대 개수.
# 'amenities',	                                # 숙소에 제공되는 편의 시설 목록.
# 'price',	                                    # 숙소의 1박당 가격.

'minimum_nights',	                            # 최소 숙박 가능 일수.
'maximum_nights',	                            # 최대 숙박 가능 일수.
'minimum_minimum_nights',	                    # 호스트가 설정한 최소 숙박 일수 중 가장 낮은 값.
'maximum_minimum_nights',	                    # 호스트가 설정한 최소 숙박 일수 중 가장 높은 값.
'minimum_maximum_nights',	                    # 호스트가 설정한 최대 숙박 일수 중 가장 낮은 값.
'maximum_maximum_nights',	                    # 호스트가 설정한 최대 숙박 일수 중 가장 높은 값.
'minimum_nights_avg_ntm',	                    # 최소 숙박 일수의 평균값.
'maximum_nights_avg_ntm',	                    # 최대 숙박 일수의 평균값.
'calendar_updated',	                            # 달력이 마지막으로 업데이트된 시점.
'has_availability',	                            # 숙소 예약 가능 여부.
'availability_30',	                            # 향후 30일 동안의 숙소 예약 가능 일수.
'availability_60',	                            # 향후 60일 동안의 숙소 예약 가능 일수.
'availability_90',	                            # 향후 90일 동안의 숙소 예약 가능 일수.
'availability_365',	                            # 향후 365일 동안의 숙소 예약 가능 일수.
'calendar_last_scraped',	                    # 달력 정보가 마지막으로 스크래핑된 날짜.

# 'number_of_reviews',	                        # 총 리뷰 개수.
# 'number_of_reviews_ltm',	                    # 지난 12개월 동안의 리뷰 개수.
# 'number_of_reviews_l30d',	                    # 지난 30일 동안의 리뷰 개수.

'availability_eoy',	                            # 연말까지의 예약 가능 일수.

# 'number_of_reviews_ly',	                        # 지난 1년 동안의 리뷰 개수.

'estimated_occupancy_l365d',	                # 지난 365일 동안의 예상 점유율.
'estimated_revenue_l365d',	                    # 지난 365일 동안의 예상 수입.

# 'first_review',	                                # 첫 번째 리뷰가 작성된 날짜.
# 'last_review',	                                # 마지막 리뷰가 작성된 날짜.
# 'review_scores_rating',	                        # 총 리뷰 점수(별점)의 평균값.
# 'review_scores_accuracy',	                    # 정확도 리뷰 점수.
# 'review_scores_cleanliness',	                # 청결도 리뷰 점수.
# 'review_scores_checkin',	                    # 체크인 경험 리뷰 점수.
# 'review_scores_communication',	                # 소통 리뷰 점수.
# 'review_scores_location',	                    # 위치 리뷰 점수.
# 'review_scores_value',	                        # 가격 대비 가치 리뷰 점수.

'license',	                                    # 숙소의 라이선스 정보.
'instant_bookable',	                            # 즉시 예약이 가능한 숙소인지 여부.
'calculated_host_listings_count',	            # 호스트가 등록한 숙소 개수.
'calculated_host_listings_count_entire_homes',	# 호스트가 등록한 '전체 숙소/아파트' 유형의 개수.
'calculated_host_listings_count_private_rooms',	# 호스트가 등록한 '개인실' 유형의 개수.
'calculated_host_listings_count_shared_rooms',	# 호스트가 등록한 '공용 공간' 유형의 개수.
'reviews_per_month'	                            # 월별 평균 리뷰 개수.
]

# 불필요 컬럼 드롭 처리
df_listings = df_listings.drop(drop_list, axis=1)



In [ ]:
# 남은 컬럼 확인
print(df_listings.columns)

_="""
neighbourhood_cleansed
latitude
longitude
property_type
room_type
accommodates
bathrooms
bedrooms
beds
amenities
price
number_of_reviews
number_of_reviews_ltm
number_of_reviews_l30d
number_of_reviews_ly
first_review
last_review
review_scores_rating
review_scores_accuracy
review_scores_cleanliness
review_scores_checkin
review_scores_communication
review_scores_location
review_scores_value
"""

In [ ]:
df_listings.info()

_="""
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25297 entries, 0 to 25296
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   neighbourhood_cleansed       25297 non-null  object 
 1   latitude                     25297 non-null  float64
 2   longitude                    25297 non-null  float64
 3   property_type                25297 non-null  object 
 4   room_type                    25297 non-null  object 
 5   accommodates                 25297 non-null  int64  
 6   bathrooms                    23253 non-null  float64       👈 결측치 있음
 7   bedrooms                     25040 non-null  float64       👈 결측치 있음
 8   beds                         23255 non-null  float64       👈 결측치 있음
 9   amenities                    25297 non-null  object 
 10  price                        23260 non-null  object        👈 결측치 있음
 11  number_of_reviews            25297 non-null  int64  
 12  number_of_reviews_ltm        25297 non-null  int64  
 13  number_of_reviews_l30d       25297 non-null  int64  
 14  number_of_reviews_ly         25297 non-null  int64  
 15  first_review                 21724 non-null  object        👈 결측치 있음
 16  last_review                  21724 non-null  object        👈 결측치 있음
 17  review_scores_rating         21724 non-null  float64       👈 결측치 있음
 18  review_scores_accuracy       21720 non-null  float64       👈 결측치 있음
 19  review_scores_cleanliness    21720 non-null  float64       👈 결측치 있음
 20  review_scores_checkin        21720 non-null  float64       👈 결측치 있음
 21  review_scores_communication  21720 non-null  float64       👈 결측치 있음
 22  review_scores_location       21720 non-null  float64       👈 결측치 있음
 23  review_scores_value          21720 non-null  float64       👈 결측치 있음
dtypes: float64(12), int64(5), object(7)
memory usage: 4.6+ MB
"""

### # 3. 결측치 및 이상치 탐색

In [ ]:
# 결측치 및 이상치 제거를 위한 사용자 정의 메서드

def fillna_groupby_mean_price(groupby_columns):
    """
    호출 예:
    groupby_columns = [
        ['room_type', 'property_type', 'accommodates', 'neighbourhood_cleansed'],
        ['room_type', 'property_type', 'accommodates'],
        ['room_type', 'accommodates'],
        ['room_type', 'property_type']
    ]
    fillna_groupby_mean_price(groupby_columns)
    """
    def get_na_cnt():
        # 'price' 열의 결측치 개수를 반환하는 내부 함수
        return df_listings['price'].isna().sum()

    print('최초 결측치', get_na_cnt(), '개 ')

    for gc in groupby_columns:
        # 각 그룹핑 컬럼을 기준으로 그룹별 평균값으로 결측치를 채움
        df_listings['price'] = df_listings['price'].fillna(
            df_listings.groupby(gc)['price'].transform('mean')
        )
        print(f"'{gc}' 컬럼으로 처리 후 결측치 {get_na_cnt()} 개")


def drop_price_above(df: pd.DataFrame, percent: float | int, price_col='price'):
    """
    주어진 DataFrame에서 특정 열의 가격이 상위 `percent`에 해당하는 값을 초과하는 행을 제거합니다.

    Args:
        df (pd.DataFrame): 처리할 원본 DataFrame.
        percent (float): 제거할 상위 백분위(예: 95는 상위 5%를 제거).
        price_col (str, optional): 가격 데이터가 있는 열의 이름. 기본값은 'price'.

    Returns:
        pd.DataFrame: 상위 `percent`에 해당하는 가격을 초과하는 행이 제거된 DataFrame.
    """
    percent = percent / 100
    rangex = df[price_col].quantile(percent)
    return df[df[price_col] < rangex]

def drop_price_over(df:pd.DataFrame ,target_price:int):
    """
    target_price(해당 금액) 이상 행 드롭해서 반환
    """
    return df.drop(df[df['price'] > target_price].index, axis=0)

In [ ]:
# 결측치 탐색

df_listings.isna().sum()

_="""
neighbourhood_cleansed            0
latitude                          0
longitude                         0
property_type                     0
room_type                         0
accommodates                      0
bathrooms                      2044     👈 결측치 있음
bedrooms                        257     👈 결측치 있음
beds                           2042     👈 결측치 있음
amenities                         0
price                          2037     👈 결측치 있음
number_of_reviews                 0
number_of_reviews_ltm             0
number_of_reviews_l30d            0
number_of_reviews_ly              0
first_review                   3573     👈 결측치 있음
last_review                    3573     👈 결측치 있음
review_scores_rating           3573     👈 결측치 있음
review_scores_accuracy         3577     👈 결측치 있음
review_scores_cleanliness      3577     👈 결측치 있음
review_scores_checkin          3577     👈 결측치 있음
review_scores_communication    3577     👈 결측치 있음
review_scores_location         3577     👈 결측치 있음
review_scores_value            3577     👈 결측치 있음
dtype: int64
"""

In [ ]:
# 추적용 price_text 컬럼 생성
df_listings['price_text'] = df_listings['price']

# 'price_float' 컬럼에서 '$'와 ',' 제거
df_listings['price'] = df_listings['price'].str.replace('$', '', regex=False).str.replace(',', '', regex=False)

# 컬럼 타입 변경
df_listings['price'] = df_listings['price'].astype(float)

In [ ]:
# price 결측치 처리를 위한 결측치가 없는 범례형 컬럼-가격 연관성 파악

rt_price = df_listings.groupby('room_type')['price'].mean().sort_values(ascending=False)
pt_price = df_listings.groupby('property_type')['price'].mean().sort_values(ascending=False)
acc_price = df_listings.groupby('accommodates')['price'].mean().sort_values(ascending=False)
nc_price = df_listings.groupby('neighbourhood_cleansed')['price'].mean().sort_values(ascending=False)


fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. room_type
sns.barplot(x=rt_price, y=rt_price.index, hue=rt_price.index, ax=axes[0, 0])
axes[0, 0].set_title('room_type')

# 2. property_type
sns.barplot(x=pt_price, y=pt_price.index, hue=pt_price.index, ax=axes[0, 1])
axes[0, 1].set_title('property_type')

# 3. accommodates
sns.barplot(x=acc_price.index, y=acc_price, hue=acc_price.index, ax=axes[1, 0])
axes[1, 0].set_title('accommodates')

# 4. neighbourhood_cleansed
sns.barplot(x=nc_price, y=nc_price.index, hue=nc_price.index, ax=axes[1, 1])
axes[1, 1].set_title('neighbourhood_cleansed')

plt.tight_layout()
plt.show()

In [ ]:
# price 결측치 그룹 별 평균값으로 채워줌

groupby_columns = [
    ['room_type', 'property_type', 'accommodates', 'neighbourhood_cleansed'],
    ['room_type', 'property_type', 'accommodates'],
    ['room_type', 'accommodates'],
    ['room_type', 'property_type']
]

fillna_groupby_mean_price(groupby_columns)

In [ ]:
# price 이상치 탐색

_="""
아래 차트로 이상치가 있음을 확인
"""

df_price_above_99 = drop_price_above(df_listings, 99)   # 상위 1% 제거
df_price_above_95 = drop_price_above(df_listings, 95)   # 상위 5% 제거
df_price_above_90 = drop_price_above(df_listings, 90)   # 상위 10% 제거
df_price_above_85 = drop_price_above(df_listings, 85)   # 상위 15% 제거


fig, axes = plt.subplots(2, 2, figsize=(12, 12))
fig.suptitle('가격 분포 비교 (상위 % 제거)', fontsize=16, fontweight='bold')

# 각 subplot에 대한 데이터와 제목
datasets = [
    (df_price_above_85, f'상위 15% 제거 ({len(df_price_above_85):,})', 0, 0, '#4472C4'),
    (df_price_above_90, f'상위 10% 제거 ({len(df_price_above_90):,})', 0, 1, '#E70012'),
    (df_price_above_95, f'상위 5% 제거 ({len(df_price_above_95):,})', 1, 0, '#70AD47'),
    (df_price_above_99, f'상위 1% 제거 ({len(df_price_above_99):,})', 1, 1, '#FFC000')
]

for i, (df, title, row, col, color) in enumerate(datasets):
    # boxplot 그리기
    sns.boxplot(
        y=df['price'],
        ax=axes[row, col],
        color=color,
        width=0.5,
        flierprops=dict(
            marker='o', markerfacecolor=color, markersize=5, 
                markeredgewidth=0.8, alpha=0.9
        )
    )
    axes[row, col].set_title(title)
   

plt.tight_layout()
plt.show()

In [ ]:
# 이상치 제거 후 적용

df_listings = df_price_above_85

### # 4. 데이터 시각화를 통한 탐색

In [ ]:
# 편의시설 분석

import ast # 리스트형 문자열 파싱용 라이브러리
amenities_ser = df_listings['amenities'].apply(ast.literal_eval)

In [ ]:
# 편의시설 개수 - price의 상관관계

df_listings['amenity_count'] = amenities_ser.apply(len)

ac_price = df_listings.groupby('amenity_count')['price'].mean().sort_values(ascending=False)

plt.figure(figsize=(20,6))
# plt.title(title)
sns.barplot(
    x=ac_price.index,
    y=ac_price.values,
    hue=ac_price
)
plt.show()

_="""

단순 분석으로는 영향도는 있으나 큰 변화는 없다고 판단.

"""

In [ ]:
# 편의시설 상세 분석 1 - 전체 갯수

# 배열로 되어 있는 모든 리스트 하나로 모음
all_amenities = amenities_ser.explode().tolist()

# 중복제거용
dictinst_amenities = set(all_amenities)

# 중복제거 후 갯수
print(len(dictinst_amenities))

_="""

총 2,524 가지의 편의시설 종류를 발견

"""

In [ ]:
# 편의시설 상세 분석 2-1,ㄴ 카테고리화 - 상세 분석 위한 파일로 저장

f1 = open('amenities.txt','w', encoding="utf-8-sig", errors='replace')

for idx,item in enumerate(dictinst_amenities):
    print(idx, item)
    item = str(item)
    f1.write(item+'\n')

f1.close()

In [ ]:
# 편의시설 상세 분석 2-2, 카테고리화 - 

amenity_keywords = {
    "TV/디스플레이": ["hdtv", "tv", "chromecast", "fire tv", "apple tv", "netflix", "hulu", "dvd player", "amazon prime video","standard cable", "premium cable", "roku"],
    "사운드": ["sound system", "speaker"],
    "냉장고/오븐/가전": ["refrigerator", "oven", "microwave", "stainless steel", "induction stove", "gas stove", "electric stove", "single oven", "double oven", "ih", "freezer"],
    "샴푸/린스/바디워시": ["shampoo", "conditioner", "body soap", "トリートメント", "リンス", "ボディソープ", "コンディショナー", "treatment", "body wash", "hair oil", "洗手液", "せっけん", "洗发水", "リンスインシャンプー"],
    "와이파이/인터넷": ["wifi", "ethernet", "pocket wifi", "lan connection"],
    "주차": ["parking", "garage", "driveway", "carport", "parking lot", "parking garage", "street parking"],
    "운동/헬스": ["exercise equipment", "gym", "yoga mat", "shared gym", "stationary bike", "workout bench", "rowing", "treadmill", "elliptical", "climbing wall", "batting cage", "pool", "sauna", "private gym"],
    "어린이/유아": ["high chair", "crib", "children’s books", "children's playroom", "baby bath", "changing table", "table corner guards", "pack ’n play", "travel crib", "booster seat", "baby monitor", "safety gates", "children’s bikes", "children’s dinnerware"],
    "주방/조리도구": ["coffee", "coffee maker", "toaster", "blender", "rice maker", "dishwasher", "keurig", "nespresso", "bbq grill", "barbecue utensils", "kitchen", "kitchenette", "cooking basics", "microwave", "oven", "stove", "hot water kettle", "bread maker", "dishes", "silverware", "baking", "dining table", "wine glass"],
    "사우나/스파/욕조": ["hot tub", "private sauna", "bathtub", "shared hot tub", "spa", "sauna", "shared sauna", "bidet", "shower gel", "hot water"],
    "뷰/전망": ["view", "courtyard view", "garden view", "city skyline view", "mountain view", "sea view", "lake view", "resort view", "bay view", "park view", "canal view", "river view", "ocean view", "harbor view", "valley view", "golf course view", "marina view", "waterfront", "resort access", "beach view", "beach access", "lake access"],
    "보안/안전": ["smoke alarm", "fire extinguisher", "first aid kit", "security camera", "window guards", "smart lock", "keypad", "lockbox", "carbon monoxide alarm", "noise decibel monitors", "lock on bedroom door", "outlet covers", "fireplace guards", "safe"],
    "난방/냉방": ["air conditioning", "central heating", "portable heater", "ceiling fan", "heating", "fan", "window ac", "radiant heating", "split type", "portable air conditioning", "indoor fireplace"],
    "세탁/건조": ["washer", "dryer", "laundromat", "laundry", "free washer", "paid washer", "free dryer", "paid dryer", "drying rack"],
    "반려동물": ["pets allowed"],
    "게임/오락": ["game", "arcade", "console", "board game", "ping pong", "bowling", "mini golf", "life size game"],
    "야외/정원": ["backyard", "outdoor", "patio", "balcony", "hammock", "sun lounger", "fire pit", "fenced", "private entrance", "private living room", "shared backyard", "shared beach access", "outdoor shower", "outdoor furniture", "outdoor playground", "sun loungers"],
    "의류/수납": ["clothing storage", "closet", "dresser", "wardrobe", "hanger"],
    "스포츠/레저": ["bike", "kayak", "hockey", "piano", "skate ramp", "boat slip"],
    "도서/문화": ["book", "reading", "record player", "movie theater", "theme room"],
    "기타": [
        "Bed linens", "ev charger", "elevator", "iron", "mosquito", "mini fridge",
        "room-darkening", "single level", "smoking", "trash compactor", "breakfast",
        "private patio", "private backyard", "private backyard – fully fenced",
        "private backyard – not fully fenced", "private patio or balcony",
        "shared patio or balcony", "shared backyard – fully fenced",
        "shared backyard – not fully fenced", "babysitter recommendations",
        "extra pillows and blankets", "long term stays allowed", "mosquito net",
        "outdoor dining area", "patio or balcony", "private entrance",
        "private living room", "room-darkening shades", "self check-in",
        "shared backyard", "shared backyard – fully fenced",
        "shared backyard – not fully fenced", "shared beach access",
        "shared beach access – beachfront", "shared patio or balcony",
        "single level home", "smoking allowed", "housekeeping", "building staff",
        "host greets you", "cleaning available", "essentials", "luggage dropoff",
        "cleaning products", "dinnerware", "babysitter", "extra pillow",
        "blanket", "long term"
    ]
}

In [ ]:
amenities_ser2 = amenities_ser.copy()


def classify_amenities(amenities_list):
    new_list = []

    # Series 객체의 반복 가능 값 반복
    for amnt in amenities_list:

        for key,value in amenity_keywords.items():
            for key_word in value:
                if key_word.lower() in amnt.lower():
                    new_list.append(key)
                    is_changed = True
                    break
        
        if not is_changed:
            new_list.append('기타')
        
    return list(set(new_list))


amenities_ser2 = amenities_ser2.apply(classify_amenities)


am3

In [ ]:
amenities_unique = set(all_amenities)

In [ ]:
len(all_amenities)

In [ ]:
# 편의시설 옵션 가지수 생성

print("=== 편의시설 옵션 가지수 생성 ===")

# 방법 1: 편의시설 총 개수 (숙소별)
df_listings['amenity_count'] = amenities_ser.apply(len)

print(f"\n1. 편의시설 개수 분포:")
print(f"   최소: {df_listings['amenity_count'].min()}개")
print(f"   최대: {df_listings['amenity_count'].max()}개") 
print(f"   평균: {df_listings['amenity_count'].mean():.1f}개")
print(f"   중앙값: {df_listings['amenity_count'].median()}개")

# 방법 2: 주요 편의시설별 더미 변수 생성 (상위 20개)
from collections import Counter

amenity_counts = Counter(all_amenities)
top_20_amenities = [amenity for amenity, count in amenity_counts.most_common(20)]

print(f"\n2. 상위 20개 편의시설 더미 변수 생성:")
for amenity in top_20_amenities:
    # 컬럼명을 깔끔하게 만들기 (공백과 특수문자 제거)
    col_name = f"has_{amenity.replace(' ', '_').replace('-', '_').replace('/', '_').lower()}"
    df_listings[col_name] = amenities_ser.apply(lambda x: 1 if amenity in x else 0)
    print(f"   {col_name}: {df_listings[col_name].sum():,}개 숙소 보유")

print(f"\n3. 생성된 컬럼들:")
amenity_cols = [col for col in df_listings.columns if col.startswith('has_')]
print(f"   총 {len(amenity_cols)}개 편의시설 더미 변수 생성")
print(f"   컬럼들: {amenity_cols[:5]}... (처음 5개만 표시)")

In [ ]:
# 편의시설 카테고리별 분류

print("=== 편의시설 카테고리별 분류 ===")

# 편의시설을 카테고리별로 분류
wifi_related = ['Wifi', 'Internet', 'Wireless Internet']
kitchen_related = ['Kitchen', 'Refrigerator', 'Microwave', 'Dishwasher', 'Coffee maker', 'Cooking basics']
bathroom_related = ['Hot water', 'Shampoo', 'Hair dryer', 'Body soap', 'Bath towel', 'Toilet paper']
comfort_related = ['Air conditioning', 'Heating', 'Fireplace', 'Fan']
entertainment_related = ['TV', 'Cable TV', 'Netflix', 'Sound system']
safety_related = ['Smoke detector', 'Carbon monoxide detector', 'Fire extinguisher', 'First aid kit']
parking_related = ['Free parking on premises', 'Paid parking on premises', 'Street parking']

categories = {
    'wifi_amenities': wifi_related,
    'kitchen_amenities': kitchen_related, 
    'bathroom_amenities': bathroom_related,
    'comfort_amenities': comfort_related,
    'entertainment_amenities': entertainment_related,
    'safety_amenities': safety_related,
    'parking_amenities': parking_related
}

# 각 카테고리별 점수 계산
for category, amenity_list in categories.items():
    df_listings[category] = amenities_ser.apply(
        lambda x: sum(1 for amenity in amenity_list if amenity in x)
    )
    print(f"{category}: 평균 {df_listings[category].mean():.1f}개")

# 총 편의시설 카테고리 점수
df_listings['total_category_score'] = sum(df_listings[cat] for cat in categories.keys())

print(f"\n4. 카테고리별 편의시설 점수:")
print(f"   총 카테고리 점수 평균: {df_listings['total_category_score'].mean():.1f}")
print(f"   카테고리별 점수 분포:")
for category in categories.keys():
    print(f"   - {category}: {df_listings[category].mean():.1f}")

# 편의시설 등급 생성 (상/중/하)
def categorize_amenities(count):
    if count >= 30:
        return 'premium'  # 프리미엄 (30개 이상)
    elif count >= 15:
        return 'standard'  # 스탠다드 (15-29개)
    else:
        return 'basic'     # 베이직 (14개 이하)

df_listings['amenity_grade'] = df_listings['amenity_count'].apply(categorize_amenities)

print(f"\n5. 편의시설 등급 분포:")
grade_counts = df_listings['amenity_grade'].value_counts()
for grade, count in grade_counts.items():
    percentage = count / len(df_listings) * 100
    print(f"   {grade}: {count:,}개 숙소 ({percentage:.1f}%)")

In [ ]:
# 편의시설 빈도 분석

from collections import Counter

print("6. 가장 인기 있는 편의시설 TOP 20")
amenity_counts = Counter(all_amenities)
top_amenities = amenity_counts.most_common(20)

for i, (amenity, count) in enumerate(top_amenities, 1):
    print(f"{i:2}. {amenity:<30} : {count:,}회")

print(f"\n7. 특정 편의시설 확인 예시")
print(f"WiFi 보유 숙소: {amenity_counts.get('Wifi', 0):,}개")
print(f"에어컨 보유 숙소: {amenity_counts.get('Air conditioning', 0):,}개")
print(f"주방 보유 숙소: {amenity_counts.get('Kitchen', 0):,}개")

In [ ]:
# 편의시설 시각화

import matplotlib.pyplot as plt

# 상위 15개 편의시설 바차트
top_15_amenities = amenity_counts.most_common(15)
amenities, counts = zip(*top_15_amenities)

plt.figure(figsize=(12, 8))
bars = plt.barh(range(len(amenities)), counts, color='skyblue', alpha=0.8)
plt.yticks(range(len(amenities)), amenities)
plt.xlabel('숙소 수')
plt.title('가장 인기 있는 편의시설 TOP 15', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()  # 상위 항목을 위쪽에 표시

# 각 바에 숫자 표시
for i, bar in enumerate(bars):
    width = bar.get_width()
    plt.text(width + 100, bar.get_y() + bar.get_height()/2, 
             f'{counts[i]:,}', ha='left', va='center')

plt.tight_layout()
plt.show()

print(f"\n8. 편의시설 개수별 숙소 분포")
amenity_lengths.value_counts().sort_index().head(10)

### # 5. 데이터 정제 및 전처리

In [ ]:
# 결측치 행 제거

_="""
아래 컬럼들의 결측치에 대해서 무조건적인 드롭을 하려 했으나
"정말 화장실, 침실이 없는 경우는 어떻게 하지?" 라는 생각이 들었음
일단 공동 작업을 위해 드롭은 하나. ML에서 평가가 낮게 나올경우 재고 하기로 함


bathrooms                      2044     👈 결측치 있음
bedrooms                        257     👈 결측치 있음
beds                           2042     👈 결측치 있음
"""

# bathrooms 결측치 있는 행 제거
df_listings = df_listings.dropna(subset=['bathrooms'])

# bedrooms 결측치 있는 행 제거
df_listings = df_listings.dropna(subset=['bedrooms'])

# beds 결측치 있는 행 제거
df_listings = df_listings.dropna(subset=['beds'])


df_listings.isna().sum()